<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# Set base parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes"
LOAD_FILE = os.path.join(LOAD_PATH, "guideline_raw.json")

SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "guideline_structured.json")

In [ ]:
# Load LLM

"""
Focus should be on instruction-following models from Hugging Face
With free licence (Apache)
Qwen seems to have been trained on producing JSON formats
...allows for many tokens as input (up to 128k!)
...parameters are good balance between small and big
Should also check Llama offerings?
"""

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [ ]:
# Test

## Should also carry out test prompt of transforming recommendations

text = "Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable?"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=500)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

In [5]:
# Load JSON of raw recommendations

def load_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {file_path}')

raw_recommendations = load_json(LOAD_FILE)

print(type(raw_recommendations))
print(len(raw_recommendations))
print(raw_recommendations[0])

<class 'list'>
136
{'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': None, 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.'}


In [6]:
def construct_prompt(entity):

    """
    Given one recommendation entry {}, creates the prompt to extract one normalised JSON item
    """

    heading_1 = entity.get('heading_1')
    sub_heading_1 = entity.get('sub_heading_1')
    sub_heading_2 = entity.get('sub_heading_2')

    original_recommendation_number = entity.get('original_recommendation_number')
    original_recommendation_text = entity.get('original_recommendation_text')

    heading_context = " > ".join(h.strip() for h in [heading_1, sub_heading_1, sub_heading_2] if isinstance(h, str) and h.strip())

    return f"""
    You are extracting structured information from a NICE guideline recommendation.

    RULES:
    - output must be valid JSON ONLY (no markdown; no commentary)
    - do not invent clinical information, thresholds or populations; use only what is present in the recommendation text
    - For ALL clinical descriptor fields (e.g. phase, severity, medication): populate a value ONLY if it is explicitly stated in the recommendation text. Do NOT infer information that is not directly stated. If not explicit, use null.
    - 'action' must be a verb phrase indicating what is being done and to/for whom (e.g., 'ensure that people have access to calming environments and reduced stimulation', not just 'ensure')
    - 'scope' must be the setting/service/context (e.g., 'in primary care', 'in secondary care', 'in a service that can…', 'when assessing…')
    - 'population' is the group the recommendation applies to (e.g., 'people with…', 'older people…', 'adults…', 'children…', 'pregnant women…', etc.)
    - Extract 'conditionality' from clauses that begin 'if...' or 'where...'
    - Extract 'prohibitions' from verb phrases including 'do not', 'must not' or 'should not'
    - Extract 'urgency' as 'True' if the text includes 'urgent', 'urgently', 'immediate' or 'immediately', otherwise 'False'
    - 'manic_episode_history' must be one of: ['none', 'one', 'multiple', null]
    - 'current_manic_phase' must be one of: ['mania', 'hypomania', 'bipolar_depression', 'mixed', 'rapid_cycling', 'euthymic', null]
    - 'mania_severity' must be one of: ['mild', 'moderate', 'severe', null]
    - 'current_psychosis' must be: ['present', 'absent', null]
    - 'diagnoses' must be one or more comorbid mental health diagnoses. If more than one diagnosis is mentioned, record all as a list of strings.
    - 'current_medication' must be a medication name or null
    - 'medication_adherence' must be one of: ['good', 'poor', null]
    - 'physical_health_longterm' must be the name of a physical disease diagnosis that affects a person for more than six months. If more than one diagnosis is mentioned, record all as a list of strings.
    - 'physical_health_recent' must be the name of a transient disease (less than six months) or physical health event from the last six months. If more than one diagnosis is mentioned, record all as a list of strings.
    - 'risk' must be one of: ['self_harm', 'risk_to_others', null]
    - 'psychological_therapy' must be one of: ['offered', null]
    - 'care_coordination' must be one of: ['current', 'offered', null]
    - you MUST use 'null' if the information for any field is not explicit in the recommendation or heading
    - if there is more than one value for any field, retain all as a list of strings

    CONTEXT: {heading_context}

    RECOMMENDATION NUMBER: {original_recommendation_number}
    RECOMMENDATION TEXT: {original_recommendation_text}

    Produce JSON with exactly these keys:
    - action
    - scope
    - population
    - conditionality
    - prohibitions
    - urgency
    - manic_episode_history
    - current_manic_phase
    - mania_severity
    - current_psychosis
    - diagnoses
    - current_medication
    - medication_adherence
    - physical_health_longterm
    - physical_health_recent
    - risk
    - psychological_therapy
    - care_coordination
    - heading_context
    - original_recommendation_number
    - original_recommendation_text
    """


In [ ]:
# Check prompt length

recommendation_for_prompt_check = raw_recommendations[24]

prompt_test = construct_prompt(recommendation_for_prompt_check)

token_count = tokenizer.encode(prompt_test)

print(f"Prompt token length: {len(token_count)}\n")

print(f"Recommendation used for prompt check: {recommendation_for_prompt_check}")

Prompt token length: 962

Recommendation used for prompt check: {'heading_1': '1.3 Assessing suspected bipolar disorder in adults in secondary care', 'sub_heading_1': None, 'sub_heading_2': '[2014]', 'original_recommendation_number': '1.3.2', 'original_recommendation_text': "When assessing suspected bipolar disorder: undertake a full psychiatric assessment, documenting a detailed history of mood, episodes of overactivity and disinhibition or other episodic and sustained changes in behaviour, symptoms between episodes, triggers to previous episodes and patterns of relapse, and family history, and assess the development and changing nature of the mood disorder and associated clinical problems throughout the person's life (for example, early childhood trauma, developmental disorder or cognitive dysfunction in later life), and assess social and personal functioning and current psychosocial stressors, and assess for potential mental and physical comorbidities, and assess the person's physic

In [7]:
def run_llm_on_entity(tokenizer, model, entity):

    """
    Call the model on a single prompt using the prompt function
    Return model response
    """

    prompt = construct_prompt(entity)

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

Nested curly braces extraction: https://til.magmalabs.io/posts/01a278bb48-extracting-json-code-with-nested-curly-braces-in-ruby-the-long-painful-way-around-with-help-from-gpt4

In [9]:
def convert_output_to_true_json(llm_response):
    """
    Takes output from run_llm_on_entity
    Turns it into a true JSON dictionary
    Checks whether it is a JSON file
    """

    llm_response = llm_response.strip()

    start = llm_response.find("{")

    if start == -1: # Where -1 is not found in .find string method
        raise ValueError("In converting_output_to_true_json function, no initial { was found in the output from the LLM.\n")

    brace_count = 0
    json_string = None

    for i in range(start, len(llm_response)):

        if llm_response[i] == "{":
            brace_count += 1
        elif llm_response[i] == "}":
            brace_count -= 1

            if brace_count == 0:

                json_string = llm_response[start:i + 1].strip()
                break

    if json_string is None:
        raise ValueError("In converting_output_to_true_json function, no closing } was found in the output from the LLM.\n")

    try:
        json_object = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        raise

    if not isinstance(json_object, dict):
        raise TypeError(f"The object created by the function convert_output_to_true_json is not a JSON object. Instead it is a {type(json_object)}.")

    return json_object

In [10]:
def validate_json(json_object, required_keys):

    """
    Checks the structure of the JSON to see whether it has the required keys
    ... and is populated with the right types of data
    """

    # Needs adjustments once the final structure of the JSON is decided on
    # But avoid adapting until this is known

    missing_keys = [k for k in required_keys if k not in json_object]
    if missing_keys:
        raise ValueError(f"Missing required keys: {missing_keys}")

    extra_keys = [k for k in json_object.keys() if k not in required_keys]
    if extra_keys:
        raise ValueError(f"Unexpected extra keys: {extra_keys}")

    for key, value in json_object.items():
        if value is None:
            continue
        if key == "urgency" and isinstance(value, bool):
            continue
        if isinstance(value, str) or isinstance(value, list):
            continue
        raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")

    return json_object

In [13]:
def orchestrate_create_json(raw_recommendations, tokenizer, model, save_file):

    compiled_recommendations = []

    errors = []

    required_keys = ['action',
                    'scope',
                    'population',
                    'conditionality',
                    'prohibitions',
                    'urgency',
                    'manic_episode_history',
                    'current_manic_phase',
                    'mania_severity',
                    'current_psychosis',
                    'diagnoses',
                    'current_medication',
                    'medication_adherence',
                    'physical_health_longterm',
                    'physical_health_recent',
                    'risk',
                    'psychological_therapy',
                    'care_coordination',
                    'heading_context',
                    'original_recommendation_number',
                    'original_recommendation_text']

    counter = 0

    for i, entity in enumerate(raw_recommendations[:]): # Remove index numbers to process full batch

        llm_output_text = run_llm_on_entity(tokenizer, model, entity)

        try:
            parsed_json = convert_output_to_true_json(llm_output_text)
            validate_json(parsed_json, required_keys)

        except Exception as e:
            print(f"Error {e} at point {i}")
            errors.append({"index": i, "error": str(e), "raw_llm_output": llm_output_text})
            continue

        compiled_recommendations.append(parsed_json)

        counter += 1

        print(f'Number of recommendations processed: {counter}')

    with open(save_file, "w", encoding="utf-8") as f:
        json.dump(compiled_recommendations, f, ensure_ascii=False, indent=2)

    print(f"Here is the list of json parsing errors: {errors}\n\n")

    return compiled_recommendations, errors

In [ ]:
orchestrate_create_json(raw_recommendations, tokenizer, model, SAVE_FILE)

Number of recommendations processed: 1
Number of recommendations processed: 2
Number of recommendations processed: 3
Number of recommendations processed: 4
Number of recommendations processed: 5
Number of recommendations processed: 6
Number of recommendations processed: 7
Number of recommendations processed: 8


In [ ]:
# For checking what caused specific JSON parsing errors

parsing_error_line = 23
parsing_error_column = str(1)
parsing_error_character = 728

guideline_structured_error_location = load_json(SAVE_FILE)

slice_ = guideline_structured_error_location[parsing_error_line][parsing_error_column]
exact_character = slice_[728]

print(f"The offending entry: {slice_}\n")
print(f"The offending character: {exact_character}")